# Local tests

In [1]:
import os
from typing import Optional
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text, update, func
from sqlalchemy.orm import sessionmaker, joinedload, Session
from sqlalchemy.dialects.postgresql import UUID

from app.models.orders import (
    OrderItems,
    OrderStatus,
    Orders,
    CurrentOrderStatus,
    DeliveryOptions,
    DeliveryStatus,
    ShippingDetails,
)

from app.models.products import (  # Assuming your models are in an 'app.models' package
    Articles,
    Brands,
    Categories,
    Colors,
    ProductCategories,
    ProductOccasionalCategories,
    Products,
    SizeSystems,
    Sizing,
    StockKeepingUnits,
    Types,
    Variants,
    ProductCurrentStatus,
    ProductStatus,
    OwnerType,
    Condition,
    ArticleStatus,
    ArticleCurrentStatus,
    TypesOfOperations,
)
from app.models.users import (
    UserAddresses,
    UserPhotos,
    Users,
    UserInfo,
    UserActivity,
    UserBasket,
    Roles,
    UserRoles,
    UserSavedItems,
    UserReviewsAndRatings,
    DataPrivacyConsents,
    RoleType,
)
from app.repositories.common import BaseMixin, BulkActionsMixin, SearchMixin
from app.schemas.users import AddressType
from app.schemas.common import PaginatedResponse
from app.schemas.products import (
    ProductCreate,
    ProductUpdate,
    VariantCreate,
    SizingCreate,
    BrandCreate,
)

NameError: name 'SpecificationRead' is not defined

In [ ]:
# Database Connection (using tunneling)
DATABASE_URL = os.getenv("DATABASE_URL")  # Get your database URL
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [ ]:
def test_database_connection():
    """Tests the database connection."""
    try:
        with SessionLocal() as db:
            db.execute(text("SELECT 1"))  # Simple query to test connection
            print("Database connection successful!")
    except Exception as e:
        print(f"Database connection failed: {e}")


def test_product_crud():
    """Tests CRUD operations for Products."""
    with SessionLocal() as db:
        # CREATE
        try:
            # Create necessary related objects
            brand = Brands(name="Test Brands")
            category = Categories(name="Test Category", is_default=False)
            color = Colors(name="Test Color")
            size_system = SizeSystems(name="Test Size System")
            product_status = ProductStatus(name=ProductCurrentStatus.Available)

            db.add_all([brand, category, color, size_system, product_status])
            db.commit()

            product = Products(
                name="Test Product",
                brand_id=brand.id,
                description="This is a test product.",
                status_code=product_status.id,
            )

            sku = StockKeepingUnits(sku_name="TESTSKU", free_articles_count=10)
            db.add(sku)
            db.commit()

            variant = Variants(
                name="Test Variant",
                product_id=product.id,
                color_id=color.id,
                sku_id=sku.id,
            )
            product.variants.append(variant)

            db.add_all([product, variant])
            db.commit()
            db.refresh(product)

            product_category = ProductCategories(
                product_id=product.id, category_id=category.id
            )
            db.add(product_category)
            db.commit()

            sizing = Sizing(
                label="Test Size",
                variant_id=variant.id,
                size_system_id=size_system.id,
            )
            db.add(sizing)
            db.commit()

            print(f"Created product with ID: {product.id}")

        except Exception as e:
            print(f"Error creating product: {e}")
            db.rollback()
            return

        # READ
        try:
            read_product = db.query(Products).filter_by(id=product.id).first()
            assert read_product is not None
            assert read_product.name == "Test Product"
            print("Read product successfully.")
        except Exception as e:
            print(f"Error reading product: {e}")
            return

        # UPDATE
        try:
            read_product.description = "Updated product description"
            db.commit()
            db.refresh(read_product)
            assert read_product.description == "Updated product description"
            print("Updated product successfully.")
        except Exception as e:
            print(f"Error updating product: {e}")
            return

        # DELETE
        try:
            db.delete(read_product)
            db.commit()
            deleted_product = db.query(Products).filter_by(id=product.id).first()
            assert deleted_product is None
            print("Deleted product successfully.")
        except Exception as e:
            print(f"Error deleting product: {e}")
            return


def test_order_crud():
    """Tests CRUD operations for Orders."""
    with SessionLocal() as db:
        # CREATE
        try:
            user = Users(login="testuser", password="testpassword")
            db.add(user)
            db.commit()

            user_address = UserAddresses(
                address_line1="Test Address",
                city="Test City",
                country="Test Country",
                postal_code="12345",
                user_id=user.id,
                address_type=AddressType.Both,
            )
            db.add(user_address)
            db.commit()

            order_status = OrderStatus(name=CurrentOrderStatus.Placed)
            db.add(order_status)
            db.commit()

            order = Orders(
                user_id=user.id,
                status_code=order_status.id,
                total_price=100.00,
                total_delivery_price=10.00,
            )
            db.add(order)
            db.commit()
            db.refresh(order)

            print(f"Created order with ID: {order.id}")

        except Exception as e:
            print(f"Error creating order: {e}")
            db.rollback()
            return

        # READ
        try:
            read_order = db.query(Orders).filter_by(id=order.id).first()
            assert read_order is not None
            assert read_order.total_price == 100.00
            print("Read order successfully.")
        except Exception as e:
            print(f"Error reading order: {e}")
            return

        # UPDATE
        try:
            read_order.total_price = 120.00
            db.commit()
            db.refresh(read_order)
            assert read_order.total_price == 120.00
            print("Updated order successfully.")
        except Exception as e:
            print(f"Error updating order: {e}")
            return

        # DELETE
        try:
            db.delete(read_order)
            db.commit()
            deleted_order = db.query(Orders).filter_by(id=order.id).first()
            assert deleted_order is None
            print("Deleted order successfully.")
        except Exception as e:
            print(f"Error deleting order: {e}")
            return


def test_order_item_crud():
    """Tests CRUD operations for OrderItems (requires a Product and Order to exist)."""
    with SessionLocal() as db:
        # CREATE
        try:
            # Make sure a Product and Order exist
            test_product_crud()  # This will create a test product
            test_order_crud()  # This will create a test order

            product = db.query(Products).filter_by(name="Test Product").first()
            order = db.query(Orders).options(joinedload(Orders.users)).first()

            if not product or not order:
                print("Product or Order not found, skipping OrderItem CRUD test.")
                return

            # Create a DeliveryOptions instance (required for ShippingDetails)
            delivery_option = DeliveryOptions(name="Standard Shipping", cost=5.00)
            article_status = ArticleStatus(name=ArticleCurrentStatus.Available)
            types_of_operation = TypesOfOperations(name="Rental")
            db.add_all([delivery_option, article_status, types_of_operation])
            db.commit()

            # Create a ShippingDetails instance
            shipping_details = ShippingDetails(
                tracking_number="1234567890",
                actual_delivery_date=datetime.now(),
                delivery_option_id=delivery_option.id,
                user_address_id=order.users.addresses[0].id,
                delivery_status=DeliveryStatus.Pending,
                is_peer_to_peer=False,
            )
            db.add(shipping_details)
            db.commit()

            article = Articles(
                article="Test Article",
                owner_type=OwnerType.Platform,
                condition=Condition.New,
                sku_id=product.variants[0].sku_id,
                status_code=article_status.id,
                types_of_operation_id=types_of_operation.id,
            )
            db.add(article)
            db.commit()

            order_item = OrderItems(
                order_id=order.id,
                article_id=article.id,
                shipping_id=shipping_details.id,
                start_date=datetime(2024, 3, 10),  # Using datetime objects
                end_date=datetime(2024, 3, 15),
                time_start="10:00",
                price=10000,
            )
            db.add(order_item)
            db.commit()
            db.refresh(order_item)

            print(f"Created OrderItem with ID: {order_item.id}")

        except Exception as e:
            print(f"Error creating OrderItem: {e}")
            db.rollback()
            return

        # READ
        try:
            read_order_item = db.query(OrderItems).filter_by(id=order_item.id).first()
            assert read_order_item is not None
            assert read_order_item.order_id == order.id
            assert read_order_item.start_date == datetime(2024, 3, 10)
            print("Read OrderItem successfully.")
        except Exception as e:
            print(f"Error reading OrderItem: {e}")
            db.rollback()
            return

        # UPDATE
        try:
            new_start_date = datetime(2024, 3, 12)
            read_order_item.start_date = new_start_date
            db.commit()
            db.refresh(read_order_item)
            assert read_order_item.start_date == new_start_date
            print("Updated OrderItem successfully.")
        except Exception as e:
            print(f"Error updating OrderItem: {e}")
            db.rollback()
            return

        # DELETE
        try:
            db.delete(read_order_item)
            db.commit()
            deleted_order_item = (
                db.query(OrderItems).filter_by(id=order_item.id).first()
            )
            assert deleted_order_item is None
            print("Deleted OrderItem successfully.")
        except Exception as e:
            print(f"Error deleting OrderItem: {e}")
            db.rollback()
            return

In [4]:
if __name__ == "__main__":
    test_database_connection()
    test_product_crud()
    test_order_crud()
    test_order_item_crud()

Database connection successful!
Created product with ID: 6965494a-7077-4d88-8c8e-f4ba5c298081
Read product successfully.
Updated product successfully.
Deleted product successfully.
Created order with ID: 60cc52d7-dddf-4c4d-9244-b18cd9515207
Read order successfully.
Updated order successfully.
Deleted order successfully.
Created product with ID: f728cb7a-1a32-4bb7-bff9-210c056df6b3
Read product successfully.
Updated product successfully.
Deleted product successfully.
Created order with ID: e742d382-65db-49b9-8dd5-2ae66f4ddfda
Read order successfully.
Updated order successfully.
Deleted order successfully.
Product or Order not found, skipping OrderItem CRUD test.


# Users

In [ ]:
def test_user_related_tables_crud():
    """
    Tests CRUD operations for User-related tables, focusing on detailed
    table structure and scenario coverage.
    """

    with SessionLocal() as db:
        # ----- CREATE OPERATIONS -----
        try:
            # CREATE USER
            user = Users(
                login="testuser",
                password="testpassword123",
                is_notificated=True,
                last_login_at=datetime.now() - timedelta(days=2),
            )
            db.add(user)
            db.commit()
            db.refresh(user)
            print(f"Created User with ID: {user.id}")

            # CREATE USER INFO
            user_info = UserInfo(
                user_id=user.id,
                first_name="Test",
                last_name="User",
                phone_number="123-456-7890",
                email="testuser@example.com",
                is_lender=True,
            )
            db.add(user_info)
            db.commit()
            db.refresh(user_info)
            print(f"Created UserInfo with ID: {user_info.id}")

            # CREATE USER ACTIVITY
            user_activity = UserActivity(
                user_id=user.id,
                total_confirmed_orders=5,
                total_canceled_orders=1,
                activity_orders=3,
                subscription_now=False,
                total_money_spent=150.50,
            )
            db.add(user_activity)
            db.commit()
            db.refresh(user_activity)
            print(f"Created UserActivity with ID: {user_activity.id}")

            # CREATE ROLE (multiple roles for demonstration)
            role1 = Roles(code="lender", name=RoleType.Lender)
            role2 = Roles(code="renter", name=RoleType.Renter)
            db.add_all([role1, role2])
            db.commit()
            db.refresh(role1)
            db.refresh(role2)
            print(f"Created Role 1 with ID: {role1.id}")
            print(f"Created Role 2 with ID: {role2.id}")

            # CREATE USER ROLES (assign multiple roles)
            user_role1 = UserRoles(user_id=user.id, role_id=role1.id)
            user_role2 = UserRoles(user_id=user.id, role_id=role2.id)
            db.add_all([user_role1, user_role2])
            db.commit()
            print(
                f"Assigned Roles '{role1.name}' and '{role2.name}' to User '{user.login}'"
            )

            # CREATE USER ADDRESSES (multiple addresses)
            user_address1 = UserAddresses(
                address_line1="123 Test St",
                city="Test City",
                country="Test Country",
                postal_code="12345",
                address_type=AddressType.Shipping,
                user_id=user.id,
            )
            user_address2 = UserAddresses(
                address_line1="456 Billing Ave",
                city="Another City",
                country="Test Country",
                postal_code="67890",
                address_type=AddressType.Billing,
                user_id=user.id,
            )
            db.add_all([user_address1, user_address2])
            db.commit()
            db.refresh(user_address1)
            db.refresh(user_address2)
            print(f"Created UserAddress 1 with ID: {user_address1.id}")
            print(f"Created UserAddress 2 with ID: {user_address2.id}")

            # CREATE USER PHOTOS (multiple photos)
            user_photo1 = UserPhotos(
                image_url="https://example.com/test_photo1.jpg", user_id=user.id
            )
            user_photo2 = UserPhotos(
                image_url="https://example.com/test_photo2.png", user_id=user.id
            )
            db.add_all([user_photo1, user_photo2])
            db.commit()
            db.refresh(user_photo1)
            db.refresh(user_photo2)
            print(f"Created UserPhoto 1 with ID: {user_photo1.id}")
            print(f"Created UserPhoto 2 with ID: {user_photo2.id}")

            # CREATE REQUIRED OBJECTS FOR OTHER RELATIONSHIPS
            brand = Brands(name="Test Brands")
            category = Categories(name="Test Category", is_default=False)
            color = Colors(name="Test Color")
            size_system = SizeSystems(name="Test Size System")
            product_status = ProductStatus(name=ProductCurrentStatus.Available)
            article_status = ArticleStatus(name=ArticleCurrentStatus.Available)
            types_of_operation = TypesOfOperations(name="Rental")

            db.add_all(
                [
                    brand,
                    category,
                    color,
                    size_system,
                    product_status,
                    article_status,
                    types_of_operation,
                ]
            )
            db.commit()

            # Create StockKeepingUnits
            sku1 = StockKeepingUnits(sku_name="TESTSKU", free_articles_count=10)
            sku2 = StockKeepingUnits(sku_name="TESTSKU2", free_articles_count=5)
            db.add_all([sku1, sku2])
            db.commit()

            # Create Product
            product1 = Products(
                name="Test Product",
                brand_id=brand.id,
                description="This is a test product.",
                status_code=product_status.id,
            )
            product2 = Products(
                name="Another Test Product",
                brand_id=brand.id,
                description="This is another test product.",
                status_code=product_status.id,
            )
            db.add_all([product1, product2])
            db.commit()

            # Create Variants (for Products)
            variant1 = Variants(
                name="Test Variant 1",
                product_id=product1.id,
                color_id=color.id,
                sku_id=sku1.id,
            )
            variant2 = Variants(
                name="Test Variant 2",
                product_id=product2.id,
                color_id=color.id,
                sku_id=sku2.id,
            )
            db.add_all([variant1, variant2])
            db.commit()

            # Create Articles
            article1 = Articles(
                article="Test Article",
                owner_type=OwnerType.Platform,
                condition=Condition.New,
                sku_id=sku1.id,
                status_code=article_status.id,
                types_of_operation_id=types_of_operation.id,
            )
            article2 = Articles(
                article="Another Test Article",
                owner_type=OwnerType.Platform,
                condition=Condition.New,
                sku_id=sku2.id,
                status_code=article_status.id,
                types_of_operation_id=types_of_operation.id,
            )
            db.add_all([article1, article2])
            db.commit()

            # CREATE USER BASKET
            if sku1 and sku2:
                user_basket1 = UserBasket(user_id=user.id, sku_id=sku1.id, count=2)
                user_basket2 = UserBasket(user_id=user.id, sku_id=sku2.id, count=1)
                db.add_all([user_basket1, user_basket2])
                db.commit()
                db.refresh(user_basket1)
                db.refresh(user_basket2)
                print(f"Created UserBasket 1 with ID: {user_basket1.id}")
                print(f"Created UserBasket 2 with ID: {user_basket2.id}")
            else:
                print(
                    "Not enough StockKeepingUnits found, skipping UserBasket creation."
                )

            # CREATE USER REVIEWS AND RATINGS
            if product1 and product2:
                user_review1 = UserReviewsAndRatings(
                    user_id=user.id,
                    product_id=product1.id,
                    rating=4,
                    review="This is a test review for product 1.",
                )
                user_review2 = UserReviewsAndRatings(
                    user_id=user.id,
                    product_id=product2.id,
                    rating=5,
                    review="Excellent product! Highly recommend.",
                )
                db.add_all([user_review1, user_review2])
                db.commit()
                db.refresh(user_review1)
                db.refresh(user_review2)
                print(f"Created UserReviewsAndRatings 1 with ID: {user_review1.id}")
                print(f"Created UserReviewsAndRatings 2 with ID: {user_review2.id}")
            else:
                print(
                    "Not enough Products found, skipping UserReviewsAndRatings creation."
                )

            # CREATE USER SAVED ITEMS
            if article1 and article2:
                user_saved_item1 = UserSavedItems(
                    user_id=user.id,
                    variant_id=variant1.id,
                )
                user_saved_item2 = UserSavedItems(
                    user_id=user.id,
                    variant_id=variant2.id,
                )
                db.add_all([user_saved_item1, user_saved_item2])
                db.commit()
                db.refresh(user_saved_item1)
                db.refresh(user_saved_item2)
                print(f"Created UserSavedItems 1 with ID: {user_saved_item1.id}")
                print(f"Created UserSavedItems 2 with ID: {user_saved_item2.id}")
            else:
                print("Not enough Variants found, skipping UserSavedItems creation.")

            # CREATE DATA PRIVACY CONSENTS
            data_privacy_consent = DataPrivacyConsents(
                user_id=user.id,
                data_usage_consent=True,
                marketing_communications_consent=False,
                other_consent=True,
                consent_date=datetime.now() - timedelta(weeks=4),
            )
            db.add(data_privacy_consent)
            db.commit()
            db.refresh(data_privacy_consent)
            print(f"Created DataPrivacyConsents with ID: {data_privacy_consent.id}")

        except Exception as e:
            print(f"Error during CREATE operations: {e}")
            db.rollback()
            return

        # ----- READ OPERATIONS -----
        try:
            # READ USER with JOINED relationships
            read_user = (
                db.query(Users)
                .options(
                    joinedload(Users.info),
                    joinedload(Users.activity),
                    joinedload(Users.user_roles).joinedload(UserRoles.roles),
                    joinedload(Users.addresses),
                    joinedload(Users.photos),
                    joinedload(Users.basket),
                    joinedload(Users.reviews_and_ratings),
                    joinedload(Users.saved_items),
                    joinedload(Users.data_privacy_consents),
                )
                .filter_by(id=user.id)
                .first()
            )

            db.merge(read_user)

            # ASSERTIONS for User and related data
            assert read_user is not None
            assert read_user.login == "testuser"
            assert read_user.info.first_name == "Test"
            assert read_user.activity.total_confirmed_orders == 5

            # Accessing Roles through user_roles
            assert len(read_user.user_roles) == 2  # User has two roles
            assert any(
                user_role.roles.name == RoleType.Lender
                for user_role in read_user.user_roles
            )
            assert any(
                user_role.roles.name == RoleType.Renter
                for user_role in read_user.user_roles
            )

            # Assertions for UserAddresses
            assert len(read_user.addresses) == 2  # User has two addresses
            assert any(
                address.address_type == AddressType.Shipping
                for address in read_user.addresses
            )
            assert any(
                address.address_type == AddressType.Billing
                for address in read_user.addresses
            )

            # Assertions for UserPhotos
            assert len(read_user.photos) == 2  # User has two photos
            assert all(isinstance(photo.image_url, str) for photo in read_user.photos)

            # Assertions for UserBasket
            assert len(read_user.basket) >= 2  # User has at least two basket items
            assert all(isinstance(item.count, int) for item in read_user.basket)

            # Assertions for UserReviewsAndRatings
            assert (
                len(read_user.reviews_and_ratings) >= 2
            )  # User has at least two reviews
            assert all(
                isinstance(review.rating, int) and 1 <= review.rating <= 5
                for review in read_user.reviews_and_ratings
            )

            # Assertions for UserSavedItems
            assert len(read_user.saved_items) >= 2  # User has at least two saved items

            # Assertions for DataPrivacyConsents
            assert read_user.data_privacy_consents.data_usage_consent is True

            print("Read User and related data successfully.")

        except Exception as e:
            print(f"Error during READ operation: {e}")
            db.rollback()
            return

        # ----- UPDATE OPERATIONS -----
        try:
            read_user.info.first_name = "Updated Test"
            read_user.activity.total_confirmed_orders += 2
            read_user.photos.append(
                UserPhotos(
                    image_url="https://example.com/new_photo.jpg", user_id=read_user.id
                )
            )

            if read_user.addresses:
                address_to_update = (
                    db.query(UserAddresses)
                    .filter_by(id=read_user.addresses[0].id)
                    .first()
                )
                address_to_update.city = "Updated City"

            # Update baskets
            if read_user.basket:
                basket_item = (
                    db.query(UserBasket).filter_by(id=read_user.basket[0].id).first()
                )  # Fetch directly
                basket_item.count = 5

            # Update reviews
            if read_user.reviews_and_ratings:
                review = (
                    db.query(UserReviewsAndRatings)
                    .filter_by(id=read_user.reviews_and_ratings[0].id)
                    .first()
                )
                review.rating = 3

            db.commit()
            db.refresh(read_user)

            assert read_user.info.first_name == "Updated Test"
            assert read_user.activity.total_confirmed_orders == 7
            assert address_to_update.city == "Updated City"
            assert len(read_user.photos) == 3
            assert basket_item.count == 5
            assert review.rating == 3

            print("Updated User and related data successfully.")

        except Exception as e:
            print(f"Error during UPDATE operation: {e}")
            db.rollback()
            return

        # ----- DELETE OPERATIONS -----
        try:

            db.delete(read_user.info)
            db.delete(read_user.activity)
            db.delete(read_user.data_privacy_consents)

            for address in read_user.addresses:
                db.query(UserAddresses).filter_by(id=address.id).delete()
            for photo in read_user.photos:
                db.query(UserPhotos).filter_by(id=photo.id).delete()
            for basket_item in read_user.basket:
                db.query(UserBasket).filter_by(id=basket_item.id).delete()
            for review in read_user.reviews_and_ratings:
                db.query(UserReviewsAndRatings).filter_by(id=review.id).delete()
            for saved_item in read_user.saved_items:
                db.query(UserSavedItems).filter_by(id=saved_item.id).delete()
            for user_role in read_user.user_roles:
                db.query(UserRoles).filter_by(id=user_role.id).delete()

            db.delete(read_user)

            db.commit()

            # ASSERTIONS for deleted objects
            deleted_user = db.query(Users).filter_by(id=user.id).first()
            assert deleted_user is None

            deleted_address = (
                db.query(UserAddresses).filter_by(id=user_address1.id).first()
            )
            assert deleted_address is None
            # ... add assertions for other deleted objects

            print("Deleted User and related data successfully.")

        except Exception as e:
            print(f"Error during DELETE operation: {e}")
            db.rollback()
            return


# ... (Other test functions) ...

In [7]:
if __name__ == "__main__":
    test_user_related_tables_crud()

Created User with ID: 56fff2c2-5ade-4207-8970-9cb41537e410
Created UserInfo with ID: af8d85ea-92b9-45aa-882b-882b1136b8c6
Created UserActivity with ID: cd1946b2-39a0-4aff-924a-83fbbd054c01
Created Role 1 with ID: d301db3e-0b29-4d89-b77d-de18d7533afa
Created Role 2 with ID: 93c12753-ea45-4805-84cb-212d44c2b1cf
Assigned Roles 'RoleType.Lender' and 'RoleType.Renter' to User 'testuser'
Created UserAddress 1 with ID: 8260984c-4828-4e81-ba17-4f856d4dc698
Created UserAddress 2 with ID: 4080cfa8-8d89-4def-8b47-8bef81a72877
Created UserPhoto 1 with ID: 7e80ac6c-3b55-4611-807e-3cd38f7bc54c
Created UserPhoto 2 with ID: e01fe22d-e9b8-45a1-aa66-ba92ec5cb8d8
Created UserBasket 1 with ID: 98d3daa0-ebe5-42bb-abf5-e5e75d054240
Created UserBasket 2 with ID: 83d85977-7b6f-4819-974f-c6c68e130288
Created UserReviewsAndRatings 1 with ID: 51dac3cd-8a72-41ee-89d1-1ee5cc385c45
Created UserReviewsAndRatings 2 with ID: 11c76f25-9001-4038-a4fe-422887fb4581
Created UserSavedItems 1 with ID: 0e42aff4-715b-44e0-bded

/var/folders/cz/4jxvc_4j2jn8zpdd258vcz7w0000gn/T/ipykernel_37941/2866777164.py:362: SAWarning: DELETE statement on table 'user_basket' expected to delete 2 row(s); 0 were matched.  Please set confirm_deleted_rows=False within the mapper configuration to prevent this warning.
  db.commit()
/var/folders/cz/4jxvc_4j2jn8zpdd258vcz7w0000gn/T/ipykernel_37941/2866777164.py:362: SAWarning: DELETE statement on table 'user_reviews_and_ratings' expected to delete 2 row(s); 0 were matched.  Please set confirm_deleted_rows=False within the mapper configuration to prevent this warning.
  db.commit()
/var/folders/cz/4jxvc_4j2jn8zpdd258vcz7w0000gn/T/ipykernel_37941/2866777164.py:362: SAWarning: DELETE statement on table 'user_saved_items' expected to delete 2 row(s); 0 were matched.  Please set confirm_deleted_rows=False within the mapper configuration to prevent this warning.
  db.commit()
/var/folders/cz/4jxvc_4j2jn8zpdd258vcz7w0000gn/T/ipykernel_37941/2866777164.py:362: SAWarning: DELETE statement 

Deleted User and related data successfully.


# Repositories

In [2]:
from app.repositories.products import (
    VariantsRepository,
    BrandsRepository,
    CategoriesRepository,
    ColorsRepository,
    StockKeepingUnitsRepository,
    SizeSystemsRepository,
    SizingRepository,
    ProductStatusRepository,
    MaterialsRepository,
)
from app.schemas.products import (
    VariantCreate,
    BrandCreate,
    CategoryCreate,
    ColorCreate,
    StockKeepingUnitCreate,
    SizeSystemCreate,
    SizingCreate,
    ProductStatusCreate,
    MaterialCreate,
)

KeyError: 'app.models.products'

In [15]:
class ProductsRepository(BaseMixin, BulkActionsMixin, SearchMixin):
    """
    Repository responsible for all database operations related to the Products model.
    """

    model = Products

    # --- Product-Specific Retrieval Methods ---

    def get_available_products(
        self,
        db_session: Session,
        category_id: Optional[UUID] = None,
        brand_id: Optional[UUID] = None,
        page: int = 1,
        page_size: int = 10,
    ) -> PaginatedResponse:
        """
        Retrieves products with available stock based on filters, using pagination.
        """
        filters = {}
        relationships = {}
        if category_id:
            relationships["categories"] = ("category_id", category_id)
        if brand_id:
            filters["brand_id"] = brand_id

        return self.paginate(
            db_session=db_session,
            page=page,
            page_size=page_size,
            filters=filters,
            relationships=relationships,
        )

    def get_product_details(
        self, db_session: Session, product_id: UUID
    ) -> Optional[Products]:
        """
        Retrieves a product with all its details (variants, sizing, etc.).
        """
        product = (
            db_session.query(self.model)
            .options(
                joinedload(self.model.variants)
                .joinedload(Variants.color)
                .joinedload(Variants.sizing)
                .joinedload(Sizing.size_system),
                joinedload(self.model.brand),
                joinedload(self.model.categories).joinedload(Categories.materials),
            )
            .filter(self.model.id == product_id, self.model.deleted_at.is_(None))
            .first()
        )
        return product

    def get_available_variants(
        self, db_session: Session, product_id: UUID
    ) -> list[Variants]:
        """
        Retrieves available variants of a product based on availability of articles.
        """
        return (
            db_session.query(Variants)
            .options(joinedload(Variants.articles))
            .join(self.model)
            .filter(
                self.model.id == product_id,
                Articles.status_code == ArticleStatus.Available,
            )
            .all()
        )

    # --- Inventory and Stock Management ---

    def get_stock_count_for_variant(self, db_session: Session, variant_id: UUID) -> int:
        """
        Calculates the stock count for a specific variant.
        """
        return (
            db_session.query(func.count(Articles.id))
            .join(StockKeepingUnits, Articles.sku_id == StockKeepingUnits.id)
            .join(Variants, Variants.sku_id == StockKeepingUnits.id)
            .filter(
                Variants.id == variant_id,
                Articles.status_code == ArticleStatus.Available,
            )
            .scalar()
        )

    def decrement_stock_for_variant(
        self, db_session: Session, variant_id: UUID, quantity: int
    ):
        """
        Decrements the stock count for a specific variant (e.g., after an order).
        """
        # Ensure atomicity for stock updates. This example assumes a simplified stock update logic.
        # You might need a more complex approach (e.g., using transactions) for a production-ready system.
        db_session.execute(
            update(StockKeepingUnits)
            .where(
                StockKeepingUnits.id == variant_id,
                StockKeepingUnits.free_articles_count >= quantity,
            )
            .values(
                free_articles_count=StockKeepingUnits.free_articles_count - quantity
            )
        )
        db_session.commit()

    # --- Product Filtering and Search ---
    def filter_products(
        self,
        db_session: Session,
        category_ids: Optional[list[UUID]] = None,
        brand_ids: Optional[list[UUID]] = None,
        color_ids: Optional[list[UUID]] = None,
        size_system_ids: Optional[list[UUID]] = None,
        price_min: Optional[float] = None,
        price_max: Optional[float] = None,
        search_term: Optional[str] = None,
        page: int = 1,
        page_size: int = 10,
    ) -> PaginatedResponse:
        """
        Provides advanced filtering and search capabilities for products.
        """
        filters = {}
        relationships = {}

        if category_ids:
            relationships["categories"] = ("category_id", category_ids)
        if brand_ids:
            filters["brand_id"] = brand_ids

        if color_ids:
            relationships["variants"] = ("color_id", color_ids)
        if size_system_ids:
            relationships["variants_sizing"] = ("size_system_id", size_system_ids)
        if price_min is not None:
            filters["price_min"] = price_min
        if price_max is not None:
            filters["price_max"] = price_max

        # Apply search if a search term is provided
        if search_term:
            search_results = self.search(
                db_session=db_session,
                search_term=search_term,
                fields=["name", "description"],
                relationships=relationships,
                fuzzy_threshold=2,
                ranking=True,
            )
            return PaginatedResponse(
                items=search_results,
                page=page,
                page_size=page_size,
                total_count=len(search_results),
            )
        else:
            return self.paginate(
                db_session=db_session,
                page=page,
                page_size=page_size,
                filters=filters,
                relationships=relationships,
            )

    # --- Product Creation and Updates ---

    def create_product(
        self, db_session: Session, product_data: ProductCreate
    ) -> Products:
        """
        Creates a new product with associated variants and other details.
        Handles database transactions and relationships.
        """
        with db_session.begin():
            # Create the product
            new_product = self.create(
                db_session, **product_data.model_dump(exclude={"variants"})
            )

            # Create variants
            for variant_data in product_data.variants:
                new_variant = self._create_variant(
                    db_session, new_product, variant_data
                )
                for sizing_data in variant_data.sizing:
                    self._create_sizing(db_session, new_variant.id, sizing_data)

            db_session.refresh(new_product)
            return new_product

    def _create_variant(
        self, db_session: Session, product: Products, variant_data: VariantCreate
    ) -> Variants:
        """
        Creates a variant associated with the given product.
        """
        # Create the variant
        new_variant = Variants(
            product_id=product.id, **variant_data.model_dump(exclude={"sizing"})
        )
        db_session.add(new_variant)
        db_session.commit()
        db_session.refresh(new_variant)
        return new_variant

    def _create_sizing(
        self, db_session: Session, variant: Variants, sizing_data: SizingCreate
    ):
        """
        Creates a sizing option for the given variant.
        """
        # Create the sizing
        new_sizing = Sizing(variant_id=variant.id, **sizing_data.model_dump())
        db_session.add(new_sizing)
        db_session.commit()

    def update_product(
        self, db_session: Session, product_id: UUID, product_data: ProductUpdate
    ) -> Products:
        """
        Updates an existing product and its related data.
        """
        with db_session.begin():
            product = self.get_by_id(db_session, product_id)
            if not product:
                raise ValueError(f"Product not found with ID: {product_id}")

            # Update the product itself
            self.update(
                db_session, product, **product_data.model_dump(exclude_unset=True)
            )

            # Update or create variants (consider using bulk_upsert for efficiency)
            self._update_or_create_variants(db_session, product, product_data.variants)

            return product

    def _update_or_create_variants(
        self, db_session: Session, product: Products, variant_data: list[VariantCreate]
    ):
        """
        Updates or creates variants for a product.
        This is a simplified example, you might need a more complex strategy for handling variant updates.
        """
        for data in variant_data:
            variant_id = data.get("id")
            if variant_id:
                # Update existing variant
                variant = db_session.query(Variants).get(variant_id)
                if variant:
                    self.update(
                        db_session, variant, **data.model_dump(exclude_unset=True)
                    )
                    self._update_or_create_sizing(db_session, variant, data.sizing)
            else:
                # Create new variant
                self._create_variant(db_session, product, data)

    def _update_or_create_sizing(
        self, db_session: Session, variant: Variants, sizing_data: list[SizingCreate]
    ):
        """
        Updates or creates sizing options for a variant.
        """
        for data in sizing_data:
            sizing_id = data.get("id")
            if sizing_id:
                # Update existing sizing
                sizing = db_session.query(Sizing).get(sizing_id)
                if sizing:
                    self.update(
                        db_session, sizing, **data.model_dump(exclude_unset=True)
                    )
            else:
                # Create new sizing
                self._create_sizing(db_session, variant, data)

In [20]:
def test_products_repository():
    """Tests the ProductsRepository."""
    with SessionLocal() as db:
        products_repository = ProductsRepository()
        brands_repository = BrandsRepository()
        categories_repository = CategoriesRepository()
        colors_repository = ColorsRepository()
        size_systems_repository = SizeSystemsRepository()
        product_status_repository = ProductStatusRepository()
        stock_keeping_units_repository = StockKeepingUnitsRepository()

        # --- Test Data Setup ---
        brand_create = BrandCreate(name="Test Brand")
        brand = brands_repository.create(db, **brand_create.model_dump()) 

        category_create = CategoryCreate(name="Test Category", is_default=False)
        category = categories_repository.create(db, **category_create.model_dump())

        color_create = ColorCreate(name="Red")
        color = colors_repository.create(db, **color_create.model_dump())

        size_system_create = SizeSystemCreate(name="US")
        size_system = size_systems_repository.create(
            db, **size_system_create.model_dump()
        )

        product_status_create = ProductStatusCreate(name=ProductCurrentStatus.Available)
        product_status = product_status_repository.create(
            db, **product_status_create.model_dump()
        )

        sku_create = StockKeepingUnitCreate(sku_name="TESTSKU123", free_articles_count=10)
        sku = stock_keeping_units_repository.create(db, **sku_create.model_dump()) 

        print(brand.id)
        print(product_status.id)

        product_create_schema = ProductCreate(
            name="Test Product",
            description="A test product.",
            instructions="Handle with care.",
            brand_id=brand.id,
            status_code=product_status.id,
            variants=[
                VariantCreate(
                    name="Test Variant 1",
                    sku_id=sku.id,
                    color_id=color.id,
                    sizing=[
                        SizingCreate(
                            label="Small",
                            measurements={"chest": 36, "waist": 30},
                            size_system_id=size_system.id,
                        )
                    ],
                )
            ],
        )

        # --- Test Create Product ---
        try:
            created_product = products_repository.create_product(
                db_session=db, product_data=product_create_schema
            )
            assert created_product is not None
            assert created_product.name == "Test Product"
            assert len(created_product.variants) == 1
            variant = created_product.variants[0]
            assert len(variant.sizing) == 1
            sizing = variant.sizing[0]
            assert sizing.label == "Small"
            print("Create product successful.")

            product_category = ProductCategories(
                product_id=created_product.id, category_id=category.id
            )
            db.add(product_category)
            db.commit()
        except Exception as e:
            print(f"Error creating product: {e}")
            db.rollback()
            return

        # --- Test Get Product Details ---
        try:
            product_details = products_repository.get_product_details(
                db_session=db, product_id=created_product.id
            )
            assert product_details is not None
            assert product_details.name == "Test Product"
            assert product_details.brand.name == "Test Brands"
            assert len(product_details.categories) == 1  # Check category association
            assert product_details.categories[0].name == "Test Category"
            assert len(product_details.variants) == 1
            print("Get product details successful.")
        except Exception as e:
            print(f"Error getting product details: {e}")
            return

        # --- Test Get Available Products (no filters) ---
        try:
            available_products = products_repository.get_available_products(
                db_session=db
            )
            assert isinstance(available_products, PaginatedResponse)
            assert len(available_products.items) > 0
            assert available_products is not None
            assert len(available_products.items) == 1
            print("Get available products successful.")
        except Exception as e:
            print(f"Error getting available products: {e}")
            return

        # --- Test Get Available Products (with category filter) ---
        try:
            available_products = products_repository.get_available_products(
                db_session=db, category_id=category.id
            )
            assert isinstance(available_products, PaginatedResponse)
            assert available_products is not None
            assert len(available_products.items) > 0
            print("Get available products (with category filter) successful.")
        except Exception as e:
            print(f"Error getting available products (with category filter): {e}")
            return

In [21]:
if __name__ == "__main__":
    test_products_repository()

ProgrammingError: (pg8000.dbapi.ProgrammingError) {'S': 'ERROR', 'V': 'ERROR', 'C': '23502', 'M': 'null value in column "brand_id" of relation "products" violates not-null constraint', 'D': 'Failing row contains (642c12ad-6275-444c-ad16-9d004efdec42, null, null, null, null, null, null, Test Brand, null, null, 2024-06-16 21:19:21.287513, 2024-06-16 21:19:21.287513, null).', 's': 'public', 't': 'products', 'c': 'brand_id', 'F': 'execMain.c', 'L': '1968', 'R': 'ExecConstraints'}
[SQL: INSERT INTO products (name, description, instructions, brand_id, clothing_size_id, clasp_type_id, size_and_fit_id, status_code, accessories_size_id, updated_at, deleted_at) VALUES (%s::VARCHAR, %s::VARCHAR, %s::VARCHAR, %s::UUID, %s::UUID, %s::UUID, %s::UUID, %s::UUID, %s::UUID, now(), %s::TIMESTAMP WITHOUT TIME ZONE) RETURNING products.id, products.created_at, products.updated_at]
[parameters: ('Test Brand', None, None, None, None, None, None, None, None, None)]
(Background on this error at: https://sqlalch